This notebook preprocesses the data.

In [2]:
import sys, os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os

import pandas as pd
import numpy as np


In [3]:
%load_ext autoreload
%autoreload 2

os.chdir('/home/adam/adam/causal_inference')

from causal_inference.data.load import *
from causal_inference.data.split import *
from causal_inference.data.impute import *
from causal_inference.data.bootstrap import *

In [4]:
SEED_VALUE = 1234
OUTCOME = 'pf_ratio_12h_24h_manual_outcome'
TRAIN_SIZE = 0.8
BOOTSTRAP_SIZE = 0.95
RAW_DATA_PATH = '/home/adam/adam/data/causal_inference/data/processed/data_guerin_rct.csv'
TRAIN_DATA_PATH = '/home/adam/adam/data/causal_inference/data/processed/guerin_12_24_train.npz'
TEST_DATA_PATH = '/home/adam/adam/data/causal_inference/data/processed/guerin_12_24_test.npz'

In [5]:
np.random.seed(SEED_VALUE)

In [6]:
df = pd.read_csv(RAW_DATA_PATH)
df.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12105 entries, 0 to 12104
Data columns (total 34 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   treated                          12105 non-null  bool   
 1   po2                              12105 non-null  float64
 2   fio2                             12105 non-null  float64
 3   peep                             12105 non-null  float64
 4   pf_ratio                         12105 non-null  float64
 5   age                              12105 non-null  float64
 6   gender                           12105 non-null  object 
 7   body_mass_index                  11156 non-null  float64
 8   diabetes                         12105 non-null  bool   
 9   acute_kidney_injury              12105 non-null  bool   
 10  cirrhosis                        12105 non-null  bool   
 11  hematologic_malignancy           12105 non-null  bool   
 12  copd              

In [ ]:
df = prepare_csv_data(df=df, outcome_name=OUTCOME, threshold=0.7)
df.info()

In [ ]:
y, t, X = csv_data_to_np(df=df, outcome_name=OUTCOME, treatment_name='treated')

In [ ]:
print(y.shape)
print(t.shape)
print(X.shape)

In [ ]:
from causal_inference.data.split import train_test_split

y_train, t_train, X_train,\
    y_test, t_test, X_test = train_test_split(y, t, X, train_size=TRAIN_SIZE)

In [ ]:
from sklearn.model_selection import train_test_split

y_train, y_test, t_train, t_test,\
    X_train, X_test = train_test_split(y, t, X, train_size=TRAIN_SIZE, random_state=SEED_VALUE)

In [ ]:
print(y_train.shape)
print(t_train.shape)
print(X_train.shape)

print(y_test.shape)
print(t_test.shape)
print(X_test.shape)

In [ ]:
X_train, X_test = impute(X_train), impute(X_test)

In [ ]:
print(np.isnan(X_train).any())
print(np.isnan(X_test).any())

In [ ]:
train = bootstrap(y_train, t_train, X_train, bootstrap_size=BOOTSTRAP_SIZE, method='train')
test = bootstrap(y_test, t_test, X_test, bootstrap_size=BOOTSTRAP_SIZE, method='test')

In [ ]:
np.savez(TRAIN_DATA_PATH, yf=train[0], t=train[1], x=train[2])
np.savez(TEST_DATA_PATH, yf=test[0], t=test[1], x=test[2])